In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import torch
import copy
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

In [2]:
### config
RANDOM_SEED = 14
SEED = 14
BATCH_SIZE = 16
NUM_WORKERS = 4
LR = 1e-3
EPOCHS = 10
WEIGHT_DECAY = 1e-2
num_epoch = 200
device = 'cuda:0'
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [3]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.sequence = [s[0] for s in data]
        self.targets = [s[1] for s in data]
        self.ids = [s[2] for s in data]

    def __len__(self):
        return len(self.sequence)

    def __getitem__(self, idx):
        return self.sequence[idx], self.targets[idx], self.ids[idx]

In [14]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.hidden_size = 7
        self.input_size = 6
        # self.embedding_size = 5
        # self.id_encoder = torch.nn.Embedding(16, self.embedding_size)
        self.conv1 = torch.nn.Conv1d(16, 16, 7, padding=0)
        # self.maxpooling = torch.nn.MaxPool1d(kernel_size=5, padding=2, stride=1)
        self.conv2 = torch.nn.Conv1d(16, 16, 7, padding=0)
        self.conv3 = torch.nn.Conv1d(16, 16, 7, padding=0)
        self.lstm = torch.nn.LSTM(
            input_size = self.input_size,
            hidden_size = self.hidden_size,
            num_layers = 4,
            batch_first = True,
            dropout = 0.1,
            bidirectional = False,
        )
        self.encoder_origin_1 = torch.nn.Linear(23, 64)
        self.encoder_cnn_1 = torch.nn.Linear(17 * 3, 128)
        self.encoder_lstm_1 = torch.nn.Linear(self.hidden_size, 32)

        self.encoder_origin_2 = torch.nn.Linear(64, 256)
        self.encoder_cnn_2 =  torch.nn.Linear(128, 512)
        self.encoder_lstm_2 = torch.nn.Linear(32, 128)

        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p = 0.1)

        self.feedforward_origin_1 = torch.nn.Linear(256, 256)
        self.feedforward_cnn_1 = torch.nn.Linear(512, 512)
        self.feedforward_lstm_1 = torch.nn.Linear(128, 128)

        self.decoder_origin_1 = torch.nn.Linear(256, 64)
        self.decoder_cnn_1 = torch.nn.Linear(512, 128)
        self.decoder_lstm_1 = torch.nn.Linear(128, 32)

        self.decoder_origin_2 = torch.nn.Linear(64, 16)
        self.decoder_cnn_2 = torch.nn.Linear(128, 32)
        self.decoder_lstm_2 = torch.nn.Linear(32, 8)

        self.decoder_origin_3 = torch.nn.Linear(16, 4)
        self.decoder_cnn_3 = torch.nn.Linear(32, 8)
        self.decoder_lstm_3 = torch.nn.Linear(8, 2)

        self.predict = torch.nn.Linear(14, 1)


    def forward(self, x):
        batch = x.size()[0]

        a = self.conv1(x)
        b = self.conv2(x)
        c = self.conv3(x)
        d = x.unsqueeze(dim = 3)

        tmp_list = []
        d_list = d.tolist()

        for itr1 in range(len(d_list)):
            for itr2 in range(len(d_list[itr1])):
                for itr3 in range(0, 23 - self.input_size + 1):
                    for itr4 in range(self.input_size - 1):
                        d_list[itr1][itr2][itr3].append(d_list[itr1][itr2][itr3 + itr4][0])
                d_list[itr1][itr2] = d_list[itr1][itr2][:itr3 + 1]

        d = torch.Tensor(d_list)
        d = d.to(device)

        for i in range(batch):
            tmp_list.append(torch.unsqueeze(torch.squeeze(self.lstm(d[i])[0][ :, -1:, :], dim=1), dim = 0))

        lstm_output = torch.cat(tmp_list, dim = 0)
        cnn = torch.cat((a, b, c), dim = 2)

        x = self.encoder_origin_1(x)
        cnn = self.encoder_cnn_1(cnn)
        lstm_output = self.encoder_lstm_1(lstm_output)

        x = self.encoder_origin_2(x)
        x = self.relu(x)
        x = self.dropout(x)
        cnn = self.encoder_cnn_2(cnn)
        cnn = self.relu(cnn)
        cnn = self.dropout(cnn)
        lstm_output = self.encoder_lstm_2(lstm_output)
        lstm_output = self.relu(lstm_output)
        lstm_output = self.dropout(lstm_output)

        x = self.feedforward_origin_1(x)
        x = self.relu(x)
        x = self.dropout(x)
        cnn = self.feedforward_cnn_1(cnn)
        cnn = self.relu(cnn)
        cnn = self.dropout(cnn)
        lstm_output = self.feedforward_lstm_1(lstm_output)
        lstm_output = self.relu(lstm_output)
        lstm_output = self.dropout(lstm_output)

        x = self.decoder_origin_1(x)
        x = self.relu(x)
        x = self.dropout(x)
        cnn = self.decoder_cnn_1(cnn)
        cnn = self.relu(cnn)
        cnn = self.dropout(cnn)
        lstm_output = self.decoder_lstm_1(lstm_output)
        lstm_output = self.relu(lstm_output)
        lstm_output = self.dropout(lstm_output)

        x = self.decoder_origin_2(x)
        x = self.relu(x)
        x = self.dropout(x)
        cnn = self.decoder_cnn_2(cnn)
        cnn = self.relu(cnn)
        cnn = self.dropout(cnn)
        lstm_output = self.decoder_lstm_2(lstm_output)
        lstm_output = self.relu(lstm_output)
        lstm_output = self.dropout(lstm_output)

        x = self.decoder_origin_3(x)
        x = self.relu(x)
        x = self.dropout(x)
        cnn = self.decoder_cnn_3(cnn)
        cnn = self.relu(cnn)
        cnn = self.dropout(cnn)
        lstm_output = self.decoder_lstm_3(lstm_output)
        lstm_output = self.relu(lstm_output)
        lstm_output = self.dropout(lstm_output)

        for_predict = torch.cat((x, cnn, lstm_output), dim=2)
        x = self.predict(for_predict)

        return torch.sigmoid(x)


In [5]:
def collate_function(data):
    tmp_data = []
    tmp_target = []
    tmp_id = []
    for i in range(len(data)):
        id_converter = []
        for itr2 in range(len(data[i][0])):
            for itr3 in range(len(data[i][0][itr2])):
                # if data[i][0][itr2][itr3] >= 14:
                #     data[i][0][itr2][itr3] = 1 
                # else: 
                #     data[i][0][itr2][itr3] = 0
                data[i][0][itr2][itr3] /= 16
                if data[i][0][itr2][itr3]< 0.05:
                    data[i][0][itr2][itr3] = 0.05
            # if data[i][1][itr2][0]  >= 14: 
            #     data[i][1][itr2][0] = 0.95
            # else: 
            #     data[i][1][itr2][0] = 0.05
            data[i][1][itr2][0] /= 16
            if data[i][1][itr2][0] < 0.05:
                data[i][1][itr2][0] = 0.05
        id_converter.append([data[i][2][0]] * 16)
        tmp_data.append(data[i][0])
        tmp_target.append(data[i][1])
        tmp_id.append([copy.deepcopy(id_converter)])
    return [torch.Tensor(tmp_data), torch.Tensor(tmp_target), torch.LongTensor(tmp_id)]

In [6]:
f = open('./dataset_next_month.pickle', 'rb')
data = pickle.load(f)
f.close()
f = open('./dataset_next_month_target.pickle', 'rb')
data_target = pickle.load(f)
f.close()

In [7]:
data_list = []
for itr1 in data_target.keys():
    data_list.append((data[str(itr1)], data_target[str(itr1)], [int(itr1) - 10000000]))
dataset = MyDataset(data_list)

train_dataset, test_dataset = torch.utils.data.random_split(dataset,[498000, 40])
trainloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = NUM_WORKERS, collate_fn = collate_function)
testloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = NUM_WORKERS, collate_fn = collate_function)

In [15]:
model = MyModel()
model = model.to(device)

In [9]:
# print(len(dataloader))
for data in iter(trainloader):
    # print(inputs)
    # print(labels)
    # print(mask)
    print(len(data[2][0]))
    print(data[2][0])

    break

1
tensor([[[50394, 50394, 50394, 50394, 50394, 50394, 50394, 50394, 50394, 50394,
          50394, 50394, 50394, 50394, 50394, 50394]]])


In [10]:
for test in trainloader:
    print(test)
    print(len(test[1][0]))
    break

[tensor([[[0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.8125],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         ...,
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500]],

        [[0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         ...,
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500]],

        [[0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.8125, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.9375, 0.9375, 1.0000,  ..., 1.0000, 1.0000, 1

In [11]:
# loss_fn = torch.nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=False)
# target = torch.randn(3, 5).softmax(dim=1)
# print(target)
# print(input)
# loss_fn(input, target)

In [12]:
writer = SummaryWriter(f'./exp_{LR}_final_{BATCH_SIZE}')

In [16]:
train_steps = 1
criterion = torch.nn.MSELoss()
# criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999), eps=1e-08, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
is_inception = False

val_loss = 0.0
for epoch in range(num_epoch):

    cur_val_loss = val_loss
    val_loss = 0.0

    train_data = tqdm(iter(trainloader))
    dataloaders = {'train' : train_data, 'val' : testloader}
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels, ids in iter(dataloaders[phase]):
            inputs = inputs.to(device)
            labels = labels.to(device)
            ids = ids.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                # Get model outputs and calculate loss
                # Special case for inception because in training it has an auxiliary output. In train
                #   mode we calculate the loss by summing the final output and the auxiliary output
                #   but in testing we only consider the final output.
                if is_inception and phase == 'train':
                    # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                    outputs, aux_outputs = model(inputs)
                    loss1 = criterion(outputs, labels)
                    loss2 = criterion(aux_outputs, labels)
                    loss = loss1 + 0.4*loss2
                else:
                    outputs = model(inputs)
                    # print(len(outputs))
                    # print(len(outputs[0]))

                    loss = torch.sum(torch.pow(torch.abs(torch.subtract(outputs, labels)),3), dim = 0)
                    loss = torch.sum(loss, dim = 0)
                    
                # print(preds)
                # print(labels.data)
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    train_steps += 1
                    if(train_steps % 100) == 0:
                        train_data.set_description(f'Epoch :{epoch}' + f' loss :{round(loss.item(), 3)}' + f' val loss :{cur_val_loss}')
                        writer.add_scalar('training_loss', loss.item(), train_steps)        

                    if(train_steps % 10000 ) == 0:
                        torch.save(model.state_dict(), f'./exp_{LR}_final_{BATCH_SIZE}/' + str(train_steps) + '.model')
                        # torch.save(optimizer.state_dict(), './exp_next_month_with_id_1e-5/optimizer_' + str(train_steps) + '.model')
                    #print('propagation')
                if phase == 'val':
                    val_loss += round(loss.item(), 1)
    scheduler.step()



Epoch :0 loss :7.595 val loss :0.0:   8%|▊         | 2337/31125 [02:23<28:25, 16.88it/s]

In [ ]:
# ./exp_next_month_id_{LR}_multi_cnn/exp_next_month_id_{LR}_multi_cnn    infer 90000 120000
# ./ exp_next_month_{LR}_multi_cnn

Error: Kernel is dead